In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder
import datetime as dt
from sklearn.model_selection import train_test_split
import sqlalchemy as sqla
import datetime as dt

In [2]:
db = sqla.create_engine('sqlite:///zillow.db')
df = pd.read_sql('SELECT * FROM zillow', db,parse_dates=['Date'])

In [3]:
df['DeltaDate'] = df['Date'].apply(lambda x: dt.datetime.now()-x)
df['DeltaDate'] = df['DeltaDate'].apply(lambda x: x.days)
df

,Bedrooms,RegionName,Date,Price,DeltaDate
0,0,Memphis,1996-01-31,75490,9321
1,0,Memphis,1996-02-29,75633,9292
2,0,Memphis,1996-03-31,75801,9261
3,0,Memphis,1996-04-30,76142,9231
4,0,Memphis,1996-05-31,76524,9200
...,...,...,...,...,...
642976,5,Braden,2019-11-30,266558,617
642977,5,Braden,2019-12-31,269833,586
642978,5,Braden,2020-01-31,271472,555
642979,5,Braden,2020-02-29,276641,526


In [4]:
enc = OrdinalEncoder()
X_df = df.drop(["Price"],axis=1)
X_enc = enc.fit_transform(X_df[["RegionName","Bedrooms","DeltaDate"]])
X_df_enc = pd.DataFrame(X_enc)
X_df_enc

,0,1,2
0,302.0,0.0,290.0
1,302.0,0.0,289.0
2,302.0,0.0,288.0
3,302.0,0.0,287.0
4,302.0,0.0,286.0
...,...,...,...
642976,47.0,5.0,4.0
642977,47.0,5.0,3.0
642978,47.0,5.0,2.0
642979,47.0,5.0,1.0


In [5]:
# Separate the features (X) from the target (y):
y = df['Price']
X = X_df_enc.copy()

In [6]:
# Split data into training & testing:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_train.shape, X_test.shape

((482235, 3), (160746, 3))

In [7]:
# First review the data
# plt.scatter(df.Date, df.Price)
# plt.xlabel('Date')
# plt.ylabel('Price in USD(1,000,000)')
# plt.gcf().set_size_inches((10, 10))  
# plt.show()

In [8]:
#df.Bedrooms.value_counts()

In [9]:
#df_counts = df.Bedrooms.value_counts()
#df_counts.plot.density()

In [10]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [11]:
# Predict() method is used to generate a prediciton
y_pred = model.predict(X_test)
print(y_pred.shape)

(160746,)


In [12]:
X_test_inverse=pd.DataFrame(enc.inverse_transform(X_test))
X_test_inverse

,0,1,2
0,White Bluff,4,9047
1,Greeneville,2,4360
2,Gordonsville,3,920
3,Shiloh,0,4269
4,Andersonville,3,8136
...,...,...,...
160741,Centerville,2,4057
160742,Five Points,3,5699
160743,Troy,2,3295
160744,Pleasant View,3,6125


In [13]:
X_test_inverse["Price Predictions"]=y_pred
X_test_inverse["Actual Price"]=y_test.values
X_test_inverse

,0,1,2,Price Predictions,Actual Price
0,White Bluff,4,9047,123075.161015,103888
1,Greeneville,2,4360,121550.903025,66765
2,Gordonsville,3,920,168996.043711,155469
3,Shiloh,0,4269,77744.130821,58908
4,Andersonville,3,8136,115480.819357,122000
...,...,...,...,...,...
160741,Centerville,2,4057,125419.972142,54166
160742,Five Points,3,5699,132354.569300,66613
160743,Troy,2,3295,126229.120268,48731
160744,Pleasant View,3,6125,126431.035629,139766


In [14]:
X_test_inverse["Difference"]=X_test_inverse["Price Predictions"]-X_test_inverse["Actual Price"]
Average=X_test_inverse["Difference"].values.mean()
Average

163.7972360488048

In [15]:
data = {
  "0": [251],
  "1": [4],
  "2": [-100]
}

#load data into a DataFrame object:
df2 = pd.DataFrame(data)

print(df2) 

     0  1    2
0  251  4 -100


In [16]:
y_test=model.predict(df2)
y_test

array([215843.79934196])

In [17]:
print(model.coef_)
print(model.intercept_)

[-1.22424374e+01  2.07808234e+04 -2.35100688e+02]
112283.2885951394


In [18]:
# import joblib
# joblib.dump(model,"Trained_Linear_Regression_Model.joblib")